## <center>Лабораторная работа № 6 'Классификация обзоров фильмов'<center>

### <center>Выполнила студентка 3 курса группы БФИ2001 Калмыкова Дарья<center>

### Цель
Обучение на датасете IMDb с помощью рекуррентной нейронной сети

### Задачи
* Ознакомиться с задачей рекуррентными нейронными сетями
* Изучить способы классификации текста
* Ознакомиться с ансамблированием сетей
* Построить ансамбль сетей, который позволит получать точность не менее 97%

### Требования
1. Найти набор оптимальных ИНС для классификации текста
2. Провести ансамблирование моделей
3. Написать функцию/функции, которые позволят загружать текст и получать 
результат ансамбля сетей
4. Провести тестирование сетей на своих текстах

In [98]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.regularizers import l2
from keras.layers import Dense, LSTM, Embedding, Conv1D, MaxPooling1D, Dropout, BatchNormalization

from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.initializers import glorot_uniform

from vecstack import stacking
from sklearn.metrics import accuracy_score
from scikeras.wrappers import KerasClassifier

In [2]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
data = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

In [3]:
max_review_length = 500
X_train = pad_sequences(X_train, maxlen=max_review_length)
X_test = pad_sequences(X_test, maxlen=max_review_length)

### Model 1

In [4]:
embedding_vecor_length = 32

model = Sequential()

model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           160000    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=64)

Epoch 1/2
391/391 [==============================] - 360s 916ms/step - loss: 0.4573 - accuracy: 0.7737 - val_loss: 0.3236 - val_accuracy: 0.8651
Epoch 2/2
391/391 [==============================] - 353s 902ms/step - loss: 0.2753 - accuracy: 0.8897 - val_loss: 0.2997 - val_accuracy: 0.8757


In [6]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 87.57%


### Model 2

In [26]:
model1 = Sequential()

model1.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model1.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model1.add(MaxPooling1D(pool_size=2))

model1.add(LSTM(100))
model1.add(Dense(1, activation='sigmoid'))

model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
model1.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 500, 32)           160000    
                                                                 
 conv1d_7 (Conv1D)           (None, 500, 32)           3104      
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 250, 32)          0         
 1D)                                                             
                                                                 
 lstm_7 (LSTM)               (None, 100)               53200     
                                                                 
 dense_7 (Dense)             (None, 1)                 101       
                                                                 
Total params: 216,405
Trainable params: 216,405
Non-trainable params: 0
________________________________________________

In [28]:
model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Epoch 1/3
391/391 [==============================] - 183s 462ms/step - loss: 0.4389 - accuracy: 0.7795 - val_loss: 0.3139 - val_accuracy: 0.8690
Epoch 2/3
391/391 [==============================] - 177s 454ms/step - loss: 0.2518 - accuracy: 0.9025 - val_loss: 0.2801 - val_accuracy: 0.8831
Epoch 3/3
391/391 [==============================] - 177s 454ms/step - loss: 0.2081 - accuracy: 0.9196 - val_loss: 0.2885 - val_accuracy: 0.8847


In [29]:
scores = model1.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 88.47%


### Model 3

In [34]:
model2 = Sequential()

model2.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model2.add(LSTM(30))

model2.add(Dense(200, activation='relu'))
model2.add(Dropout(0.5))

model2.add(Dense(100, activation='relu'))
model2.add(Dropout(0.3))

model2.add(Dense(50, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [35]:
model2.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 500, 32)           160000    
                                                                 
 lstm_11 (LSTM)              (None, 30)                7560      
                                                                 
 dense_15 (Dense)            (None, 200)               6200      
                                                                 
 dropout_5 (Dropout)         (None, 200)               0         
                                                                 
 dense_16 (Dense)            (None, 100)               20100     
                                                                 
 dropout_6 (Dropout)         (None, 100)               0         
                                                                 
 dense_17 (Dense)            (None, 50)              

In [36]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=4, batch_size=64)

Epoch 1/4
391/391 [==============================] - 87s 217ms/step - loss: 0.4049 - accuracy: 0.8060 - val_loss: 0.2919 - val_accuracy: 0.8780
Epoch 2/4
391/391 [==============================] - 83s 213ms/step - loss: 0.2494 - accuracy: 0.9033 - val_loss: 0.2896 - val_accuracy: 0.8789
Epoch 3/4
391/391 [==============================] - 78s 200ms/step - loss: 0.2114 - accuracy: 0.9172 - val_loss: 0.2950 - val_accuracy: 0.8772
Epoch 4/4
391/391 [==============================] - 81s 206ms/step - loss: 0.1933 - accuracy: 0.9258 - val_loss: 0.3418 - val_accuracy: 0.8734


In [37]:
scores = model2.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 87.34%


### Model 4

In [49]:
model3 = Sequential()

model3.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))

model3.add(Dense(250, activation='selu', kernel_initializer=glorot_uniform(), bias_regularizer=l2(0.05)))
model3.add(Dropout(0.5))
model3.add(LSTM(50))

model3.add(Dense(110, activation='selu', kernel_initializer=glorot_uniform(), bias_regularizer=l2(0.05)))
model3.add(Dropout(0.3))

model3.add(Dense(50, activation='selu', kernel_initializer=glorot_uniform(), bias_regularizer=l2(0.05)))
model3.add(Dense(1, activation='sigmoid'))

model3.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [50]:
model3.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, 500, 32)           160000    
                                                                 
 dense_35 (Dense)            (None, 500, 250)          8250      
                                                                 
 dropout_15 (Dropout)        (None, 500, 250)          0         
                                                                 
 lstm_16 (LSTM)              (None, 50)                60200     
                                                                 
 dense_36 (Dense)            (None, 110)               5610      
                                                                 
 dropout_16 (Dropout)        (None, 110)               0         
                                                                 
 dense_37 (Dense)            (None, 50)              

In [51]:
model3.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=64)

Epoch 1/2
391/391 [==============================] - 319s 808ms/step - loss: 0.4567 - accuracy: 0.7769 - val_loss: 0.3780 - val_accuracy: 0.8447
Epoch 2/2
391/391 [==============================] - 394s 1s/step - loss: 0.2825 - accuracy: 0.8867 - val_loss: 0.3014 - val_accuracy: 0.8803


In [52]:
scores = model3.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 88.03%


In [100]:
model.save('model.h5')
model1.save('model1.h5')
model2.save('model2.h5')
model3.save('model3.h5')

In [94]:
# preds = model.predict(X_test, verbose=1)
# preds1 = model1.predict(X_test, verbose=1)
# preds2 = model2.predict(X_test, verbose=1)
# preds3 = model3.predict(X_test, verbose=1)
# final_preds = 0.2*preds + 0.35*preds1 + 0.15*preds2 + 0.3*preds3

### Stacking as Ensemble Method

In [95]:
# final_model = VotingClassifier(estimators=[('m', model), ('m1', model1), ('m2', model2), ('m3', model3)], voting='hard')
# final_model.fit(X_train, y_train)
# final_model.score(X_test, y_test)

In [74]:
xtrain = X_train.reshape(-1, 1)
xtest = X_test.reshape(-1, 1)
ytrain = y_train.reshape(-1, 1)

In [75]:
all_models = [model, model1, model2, model3]
s_train, s_test = stacking(all_models, xtrain, xtest, ytrain, regression=False, 
                           needs_proba=False, metric=accuracy_score, 
                           n_folds=4, stratified=True, shuffle=True, random_state=42, verbose=1)

C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


task:         [classification]
n_classes:    [4998]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [4]

model  0:     [Sequential]


C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


 72499/292969 [======>.......................] - ETA: 12:39 - loss: -461285.1875 - accuracy: 0.0019

KeyboardInterrupt: 

In [ ]:
# initializing the second-level model
final_model = model1
final_model = final_model.fit(s_train, y_train)

In [ ]:
pred_final = final_model.predict(X_test)
print(mean_squared_error(y_test, pred_final))